In [2]:
%pip install datasets trl peft bitsandbytes sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.0/138.0 kB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 kB 111.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 36.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.2/245.2 kB 188.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.1/199.1 kB 83.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 133.9 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 123.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.4/302.4 kB 104.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 74.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 172.0/172.0 kB 149.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.2/401.2 kB 145.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━

In [3]:
from huggingface_hub import login
login(token = "")

/usr/local/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


### IMPORTS

In [7]:
import os
import gc
import torch

import transformers
from transformers import AutoModelForCausalLM, TrainingArguments, BitsAndBytesConfig, AutoTokenizer, AutoModelForSeq2SeqLM
from datasets import load_dataset
from trl import DPOTrainer
import bitsandbytes as bnb

model_name = "sdevare/qgelm"

### PREPARE DATA

In [8]:
def preprocess_data(data):
    return {
        "prompt": data["event_text"] + " <SEP> " + data["question"],
        "chosen": data["accepted"],
        "rejected": data["rejected"],
    }

# Load dataset
dataset = load_dataset("sdevare/qgelm-ranks")['train']

# Save columns
original_columns = dataset.column_names

# Tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

# Format dataset
dataset = dataset.map(
    preprocess_data,
    remove_columns=original_columns
)

Generating train split: 100%|██████████| 9463/9463 [00:00<00:00, 25212.42 examples/s]
You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Map: 100%|██████████| 9463/9463 [00:01<00:00, 9013.83 examples/s]


In [9]:
dataset[0]

{'rejected': "''it like an exercise",
 'prompt': 'early-morning traffic jams beginning to develop around spring training camps<TUP> baseball organizations in florida and in arizona<TUP> a new group appeared on the scene<TUP> they nt meeting with players<TUP> they instead interviewing nonuniformed members of teams<TUP> the lawyers nt added to the clubhouse congestion<TUP> ariz. scottsdale on friday<TUP> players met with major league baseball s security traveling team and with representatives of the baseball assistance team<TUP> the meetings scheduled for 8:30<TUP> jim martin executive director of the assistance team<TUP> the same two groups at the diamondbacks  camp<TUP> security arriving to set up their material<TUP> another meeting added in this year<TUP> marvin miller his first spring training as the union executive director in 1967<TUP> don fehr on his 30th tour<TUP> kevin hallinan baseball senior vice president for security<TUP> fehr and his aides provide information the players wa

### REWARD MODEL TRAINING

In [22]:
import random
import pandas as pd
from operator import itemgetter
import torch
import warnings
warnings.filterwarnings('ignore')
from datasets import Dataset, load_dataset
from transformers import AutoModelForSequenceClassification,AutoTokenizer,TrainingArguments,T5ForSequenceClassification
from trl import RewardTrainer

In [23]:
def formatting_func(examples):
    kwargs = {"padding": "max_length", "truncation": True, "max_length": 512, "return_tensors": "pt"}
    prompt_plus_chosen_response = examples["prompt"] + "\n" + examples["chosen"]
    prompt_plus_rejected_response = examples["prompt"] + "\n" + examples["rejected"]
    tokens_chosen = tokenizer.encode_plus(prompt_plus_chosen_response, **kwargs)
    tokens_rejected = tokenizer.encode_plus(prompt_plus_rejected_response, **kwargs)
    return {
        "input_ids_chosen": tokens_chosen["input_ids"][0], "attention_mask_chosen": tokens_chosen["attention_mask"][0],
        "input_ids_rejected": tokens_rejected["input_ids"][0], "attention_mask_rejected": tokens_rejected["attention_mask"][0]
    }
formatted_dataset = dataset.map(formatting_func)

Map: 100%|██████████| 9463/9463 [00:20<00:00, 465.12 examples/s]


In [25]:
# Model to fine-tune
model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
)

Some weights of T5ForSequenceClassification were not initialized from the model checkpoint at sdevare/qgelm and are newly initialized: ['classification_head.dense.bias', 'classification_head.dense.weight', 'classification_head.out_proj.bias', 'classification_head.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [32]:
# Configuring the training arguments
training_args = TrainingArguments(
    output_dir="./reward_model",
    per_device_train_batch_size=4,
    gradient_accumulation_steps=4,
    learning_rate=5e-6,
    logging_steps=1,
    num_train_epochs = 6,
)
# Loading the RewardTrainer from TRL
trainer = RewardTrainer(
    model=model,
    args=training_args,
    tokenizer=tokenizer,
    train_dataset=formatted_dataset,
)


Detected kernel version 4.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


In [ ]:
trainer.train()

In [ ]:
from huggingface_hub import HfApi

api = HfApi()
api.upload_folder(
    folder_path="epoch-6-check-2500",
    path_in_repo="",
    repo_id="sdevare/qgelm-reward-model"
)